## Attempt for solving navier-stoke equation using PINNS

let us take the following PDE 
$$
   
    \frac{\partial u}{\partial t} 
    + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y}
    = 
    -\frac{1}{\rho} \frac{\partial p}{\partial x} 
    + \nu \left( 
    \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}
    \right) \\[10pt]

  
    \frac{\partial v}{\partial t} 
    + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y}
    = 
    -\frac{1}{\rho} \frac{\partial p}{\partial y} 
    + \nu \left( 
    \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2}
    \right)
    \\[10pt]
   
    \left( \frac{\partial u}{\partial x} \right)^2 +
    2 \left( \frac{\partial u}{\partial y} \right)
    \left( \frac{\partial v}{\partial y} \right) +
    \left( \frac{\partial v}{\partial y} \right)^2
    = -\frac{1}{\rho} 
    \left(
    \frac{\partial^2 p}{\partial x^2} 
    + \frac{\partial^2 p}{\partial y^2}
    \right)
    
$$


The boundary conditions for the velocity are the following.
$$
u(t, x, l_y) = 1, \quad u(t, x, 0) = 0, \quad
u(t, 0, y) = 0, \quad u(t, l_x, y) = 0
$$
$$
v(t, x, l_y) = 0, \quad v(t, x, 0) = 0, \quad
v(t, 0, y) = 0, \quad v(t, l_x, y) = 0
$$

consider $l_x = l_y = 1$

The boundary condition for the pressure
$$
\frac{\partial p}{\partial x}\Big|_{x=0} = 0, \quad 
\frac{\partial p}{\partial x}\Big|_{x=l_x} = 0, \quad 
\frac{\partial p}{\partial y}\Big|_{y=0} = 0, \quad 
p(t, x, l_y) = 0
$$

### PINNs attempt

We are going to train a neural network on the boundary conditions and the pde as constraints to find $u, v \& p$ 

The neural networkd is going to take 3 inputs (time, x, y) and produces 3 output (u, v, p), thus we need to initialize the inputs with the right dimensions

We are going to have $t \in [0,0.1]$ since we only want to see how system evolves in a small time step

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from matplotlib import pyplot as plt

In [117]:
def inputs(train=10, test=50):
    x_train = torch.linspace(0, 1, train, requires_grad=True).reshape(-1, 1, 1)
    y_train = torch.linspace(0, 1, train, requires_grad= True).reshape(1, -1, 1)
    t_train = torch.linspace(0, 0.1, train, requires_grad=True).reshape(1, 1, -1)

    x_test = torch.linspace(0, 1, test, requires_grad=True).reshape(-1, 1, 1)
    y_test = torch.linspace(0, 1, test, requires_grad=True).reshape(1, -1, 1)
    t_test = torch.linspace(0, 0.1, test, requires_grad=True).reshape(1, 1, -1)

    # Use torch.meshgrid to create 3D grids for training and testing
    x_grid_train, y_grid_train, t_grid_train = torch.meshgrid(x_train.squeeze(), y_train.squeeze(), t_train.squeeze())
    x_grid_test, y_grid_test, t_grid_test = torch.meshgrid(x_test.squeeze(), y_test.squeeze(), t_test.squeeze())

    # Reshape the grids to be consistent with the original code


    train_dict = {
        'x': x_grid_train,
        'y': y_grid_train,
        't': t_grid_train
    }

    test_dict = {
        'x': x_grid_test,
        'y': y_grid_test,
        't': t_grid_test
    }

    return train_dict, test_dict



In [154]:
# the model
class FCN(nn.Module):
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        self.fcs = nn.Sequential(*[
            nn.Linear(N_INPUT, N_HIDDEN),
            activation()

        ])


        self.fch =nn.Sequential(*[ nn.Sequential(*[
            nn.Linear(N_HIDDEN, N_HIDDEN),
            activation()

        ]) for _ in range(N_LAYERS-1)])
        self.fce = nn.Linear(N_HIDDEN,N_OUTPUT)
        def weights_initialization(self):
                """"
                When we define all the modules such as the layers in '__init__()'
                method above, these are all stored in 'self.modules()'.
                We go through each module one by one. This is the entire network,
                basically.
                """
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight,gain=1.0)
                nn.init.constant_(m.bias, 0)
    def forward(self, x, y, t):
        inputs = torch.cat((x, y, t), 1)
        inputs = self.fcs(inputs)
        inputs = self.fch(inputs)
        output = self.fce(inputs)
        
        # Split the output into u, v, and p
        u, v, p = torch.split(output, 1, dim=1)
        

        return u, v, p

In [143]:
def pde_loss(x,y,t,net,rho=1, train = 10):
    u, v, p = net(x,y,t) 
    u_x = torch.autograd.grad(u, x,torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x),create_graph=True)[0]
    u_y = torch.autograd.grad(u, y,torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y,torch.ones_like(u_y), create_graph=True)[0]
    u_t = torch.autograd.grad(u, t, torch.ones_like(u), create_graph=True)[0]

    v_x = torch.autograd.grad(v, x,torch.ones_like(v), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, torch.ones_like(v_x),create_graph=True)[0]
    v_y = torch.autograd.grad(v, y,torch.ones_like(v), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y,torch.ones_like(v_y), create_graph=True)[0]
    v_t = torch.autograd.grad(v,t,torch.ones_like(v_y), create_graph=True)[0]

    p_x = torch.autograd.grad(p,x,torch.ones_like(p), create_graph=True)[0]
    p_xx = torch.autograd.grad(p_x,x,torch.ones_like(p_x), create_graph=True)[0]
    p_y = torch.autograd.grad(p,y,torch.ones_like(p), create_graph=True)[0]
    p_yy = torch.autograd.grad(p_y,y,torch.ones_like(p_y), create_graph=True)[0]


    # Compute PDE losses
    loss1 = u_t + u * u_x + v * u_y + (1/rho) * p_x - v * (u_xx + u_yy)
    loss2 = v_t + u * v_x + v * v_y + (1/rho) * p_y - v * (v_xx + v_yy)
    loss3 = (u_x)**2 + 2 * u_y * v_y + (v_y)**2 + (1/rho) * (p_xx + p_yy)

    # Store losses in a dictionary
    losses = {
        'PDE1': loss1,
        'PDE2': loss2,
        'PDE3': loss3
    }
    loss_added = loss1+loss2+loss3


    return losses, loss_added
    

### setting the boundary conditions 
We are going to set the boundary conditions for neuman and dirchlit as the following

In [144]:
def boundaries_dict(train = 10, test = 50):
    y_up = torch.ones((train**3,1),requires_grad=True)
    y_down = torch.zeros((train**3,1),requires_grad=True)
    x_right = torch.ones((train**3,1),requires_grad=True)
    x_left = torch.zeros((train**3,1),requires_grad=True)

    boundaries = {
        'y_up': y_up,
        'y_down': y_down,
        'x_left': x_left,
        'x_right': x_right
    }

    return boundaries

### building the NN

We are going to build NN with 3 inputs and 3 outpus as we mentioned before

In [162]:
pinn = FCN(3,3,64,4)
mse_cost_function = torch.nn.MSELoss()
optimizer = optim.Adam(pinn.parameters(), lr=1e-3)
MSE_loss = []
iteration = []
boundary_loss = []
train_dict, test_dict = inputs()
boundaries = boundaries_dict()

def iterations(train=10, test = 50):

    
    for i in range(5001):
        optimizer.zero_grad()

        #Boundary loss, 1- top "y=1"

        u, v , p = pinn(train_dict['x'].reshape(-1,1),boundaries['y_up'].reshape(-1,1),
                         train_dict['t'].reshape(-1,1))
        loss1 = mse_cost_function(u, torch.ones((train**3,1)))
        loss2 = mse_cost_function(v, torch.zeros((train**3,1)))
        loss3 = mse_cost_function(p, torch.zeros((train**3,1)))

        #Boundary loss, 2 - down "y=0"
        u, v , p = pinn(train_dict['x'].reshape(-1,1),boundaries['y_down'].reshape(-1,1),
                         train_dict['t'].reshape(-1,1))
        loss4 = mse_cost_function(u, torch.zeros((train**3,1)))
        loss5 = mse_cost_function(v, torch.zeros((train**3,1)))
        p_y = torch.autograd.grad(p,boundaries['y_down'].reshape(-1,1), torch.ones_like(p), create_graph=True)[0]
        loss6 = mse_cost_function(p_y, torch.zeros((train**3,1)))      

        # Boundary loss, 3 - left "x=0" 
        u, v , p = pinn(boundaries['x_left'].reshape(-1,1),train_dict['y'].reshape(-1,1),
                         train_dict['t'].reshape(-1,1))
        loss7 = mse_cost_function(u, torch.zeros((train**3,1)))
        loss8 = mse_cost_function(v, torch.zeros((train**3,1)))
        p_x = torch.autograd.grad(p,boundaries['x_left'].reshape(-1,1), torch.ones_like(p), create_graph=True)[0]
        loss9 = mse_cost_function(p_x, torch.zeros((train**3,1))) 


        # Boundary loss, 4 - right "x=1" 
        u, v , p = pinn(boundaries['x_right'].reshape(-1,1),train_dict['y'].reshape(-1,1),
                         train_dict['t'].reshape(-1,1))
        loss10 = mse_cost_function(u, torch.zeros((train**3,1)))
        loss11 = mse_cost_function(v, torch.zeros((train**3,1)))
        p_x = torch.autograd.grad(p,boundaries['x_right'].reshape(-1,1), torch.ones_like(p), create_graph=True)[0]
        loss12 = mse_cost_function(p_x, torch.zeros((train**3,1))) 

        # pde loss

        pde_losses, losses_added = pde_loss(x = train_dict['x'], y = train_dict['y'], t = train_dict['t'], net = pinn  )
        loss13 = mse_cost_function(losses_added, torch.zeros((train**3,1)))
        loss = loss1+loss2+loss3+loss4+loss5+loss6+loss7+loss8+loss9+loss10+loss11+loss12+loss13

        # back propagation

        loss.backward()
        optimizer.step()


In [163]:
iterations()

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.